# Which TESS candidates should I target?
Use this notebook to filter a list of TESS Objects of Interst (TOI) for visibility at a specific site and time and for effective temperature. This template checks for visibility from the Calar Alto Observatory (CAHA) in Spain, but it is easy to adapt to a different use case.

#### Minimum Working Instructions:
- download a CSV file containing TESS alerts (from tev.mit.edu) and save it in `data/`
- Adjust the parameters (see below) to your needs
- execute the cells in this notebook 

In [ ]:
import astroplan
import numpy as np
import datetime as dt
from astroquery.mast import Catalogs
from astroplan.plots import plot_airmass
from observability import *
from IPython.core.display import display, HTML

apply some convenient ipython magic

In [ ]:
%load_ext autoreload
%autoreload 2

display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_columns', 500)
pd.set_option('precision',2)

## Specify Parameters
Define observational constraints, the Calar Alto Observatory, a maximum effective temperature.

In [ ]:
alertsFile = 'data/csv-file-toi-catalog.csv' # path to the alerts file
lastUpdate = '2021-01-25'                 # include only alerts added after this date
earliestObs = str(np.datetime64('today')) # earliest time we are interested in for observations
latestObs = '2021-12-31 12:00'            # latest time we are interested in for observations
minAltitude = 30                          # minimum elevation for visibility [deg]
Teff_max = 4000                           # maximum temperature for the M-dwarf-filter [K]
CAHA = defineCAHA()                       # define observing location, in this case Calar Alto observatory
Jmax = 12.                                # maximum J magnitude

## Read alerts
Specify minimum time in column "Updated" (=last time we checked)

In [ ]:
# alerts, targets = targetsFromCSV(alertsFile, minUpdated=lastUpdate)
alerts, targets = targetsFromCSV(alertsFile, minUpdated=lastUpdate, header=4)
alerts.head()
print(len(alerts))

## Define Observing Constraints

In [ ]:
constraints, start_time, end_time = define_constraints(minAltitude, earliestObs, latestObs)

## Check Observability from CAHA Given the Constraints

In [ ]:
# The IERS server was down. Increase auto-download max age parameter
from astropy.utils import iers
iers.conf.auto_max_age = 60.

In [ ]:
%%time
observables = check_observability(alerts, constraints, CAHA, targets, Time(earliestObs),
                                 Time(latestObs))
print(len(observables))

## Filter for M dwarf Spectral Type

Query TIC on MAST with observable objects, simultaneously filtering by effective temperature and J magnitude

In [ ]:
%%time
observableMdwarfs = Catalogs.query_criteria(catalog="Tic",Teff=[0,Teff_max], Jmag=[-np.inf, Jmax], 
                                            ID=observables.TIC).to_pandas()
print(len(observableMdwarfs))

## Output
Display the information we need in a convenient way and produce plots that facilitate target selection.

Merge the two tables

In [ ]:
observableMdwarfs.rename(columns={'ID' : 'TIC'}, inplace=True)
observableMdwarfs.TIC = observableMdwarfs.TIC.astype('int64')
observableMdwarfs = pd.merge(observables, observableMdwarfs, on='TIC', how='inner')
candidateTargets = [t for t in targets if float(t.name) in observableMdwarfs['TOI'].unique()]
len(observableMdwarfs)

Display columns relevant for candidates spreadsheet, and write them to an xlsx file in `output/`.

In [ ]:
to_spreadsheet = observableMdwarfs[['TIC', 'TOI', 'GAIA', 'Transit Depth Value', 'TIC Right Ascension', 'TIC Declination', 'rad', 'mass','Teff', 'logg', 'Orbital Period Value', 'Transit Duration Value','gmag', 'Jmag', 'Hmag', 'Kmag']]

with pd.ExcelWriter('output/newCandidates.xlsx', engine='openpyxl', float_format="%.20f") as writer:
    to_spreadsheet.to_excel(writer)
to_spreadsheet

# Plots

Print StarAlt-suited coordinate output for candidates

In [ ]:
coordList = [print('{} {:.4f} {:.4f}'.format(o.TIC, o['TIC Right Ascension'], o['TIC Declination'])) for i, o in observableMdwarfs.iterrows()]

check elevation at CAHA of the targets _tonight_

In [ ]:
fig, ax = plt.subplots()
ax = plot_airmass(candidateTargets[:], CAHA, str(dt.datetime.now().replace(hour=23, minute=59)), brightness_shading=True,
             altitude_yaxis=True, max_airmass=2.25)
ax.legend(facecolor='lightgray')
ax.grid(False, axis='y')
ax.axhspan(2, 3, alpha=.4, facecolor='w', edgecolor='maroon', hatch='//', linestyle='--')
plt.show()

In [ ]:
fig.savefig('output/airmass.png', dpi=300)

visualize long-term (next few months) visibility of candidates

In [ ]:
fig, ax = plot_observability(candidateTargets, constraints, CAHA, Time(earliestObs), Time(latestObs))

In [ ]:
fig.savefig('output/observability.png', dpi=300)

------------------------------